### Test de Demucs

In [1]:
from pathlib import Path
import torch
import librosa
import soundfile as sf
from demucs.pretrained import get_model
from demucs.apply import apply_model

In [2]:
# ---------- PATHS ----------
INPUT_PATH = Path("data/input/Hello - ssymon edit.mp3")
OUTPUT_DIR = Path("data/output_demucs_2")

OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"👉 Fichier d'entrée : {INPUT_PATH.resolve()}")
print(f"👉 Dossier de sortie : {OUTPUT_DIR.resolve()}")

# ---------- CHARGEMENT DU MODÈLE ----------
print("⏳ Chargement du modèle Demucs (htdemucs)...")
model = get_model("htdemucs")

# ---------- LECTURE DE L'AUDIO (librosa AU LIEU de torchaudio) ----------
print("⏳ Chargement de l'audio avec librosa...")
waveform, sr = librosa.load(str(INPUT_PATH), sr=None, mono=False)

# librosa retourne (time) ou (channels, time)
# → on force le format [channels, time]
import numpy as np
if waveform.ndim == 1:
    waveform = np.expand_dims(waveform, axis=0)

# Convertir en tenseur torch : [1, channels, time]
waveform = torch.tensor(waveform).float().unsqueeze(0)

# ---------- SÉPARATION ----------
print("🎧 Séparation en stems...")
sources = apply_model(model, waveform)[0] # [stems, channels, time]

# ---------- SAUVEGARDE avec soundfile ----------
for source_tensor, name in zip(sources, model.sources):
    out_path = OUTPUT_DIR / f"{INPUT_PATH.stem}_{name}.wav"
    sf.write(str(out_path),
             source_tensor.cpu().numpy().T,  # transpose → (time, channels)
             sr)
    print(f"✅ Stem '{name}' sauvegardé : {out_path.resolve()}")


👉 Fichier d'entrée : /Users/theodorefischer/Desktop/ENSTA/3A/Projet IA/repo/Projet_IA_2526/data/input/Hello - ssymon edit.mp3
👉 Dossier de sortie : /Users/theodorefischer/Desktop/ENSTA/3A/Projet IA/repo/Projet_IA_2526/data/output_demucs_2
⏳ Chargement du modèle Demucs (htdemucs)...
⏳ Chargement de l'audio avec librosa...
🎧 Séparation en stems...
✅ Stem 'drums' sauvegardé : /Users/theodorefischer/Desktop/ENSTA/3A/Projet IA/repo/Projet_IA_2526/data/output_demucs_2/Hello - ssymon edit_drums.wav
✅ Stem 'bass' sauvegardé : /Users/theodorefischer/Desktop/ENSTA/3A/Projet IA/repo/Projet_IA_2526/data/output_demucs_2/Hello - ssymon edit_bass.wav
✅ Stem 'other' sauvegardé : /Users/theodorefischer/Desktop/ENSTA/3A/Projet IA/repo/Projet_IA_2526/data/output_demucs_2/Hello - ssymon edit_other.wav
✅ Stem 'vocals' sauvegardé : /Users/theodorefischer/Desktop/ENSTA/3A/Projet IA/repo/Projet_IA_2526/data/output_demucs_2/Hello - ssymon edit_vocals.wav


In [ ]:
import importlib.metadata as md

pkgs = ["torch", "torchaudio", "torchvision", "demucs", "librosa", "soundfile", "numpy", "scipy"]

with open("requirements.txt", "w") as f:
    for p in pkgs:
        try:
            v = md.version(p)
            line = f"{p}=={v}"
            print(line)
            f.write(line + "\n")
        except md.PackageNotFoundError:
            print(f"⚠️ {p} non installé dans cet env")
